In [1]:
import glob
import os
import math

import cv2 as cv
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Conv2DTranspose
from tensorflow.keras.callbacks import ModelCheckpoint

import matplotlib.pyplot as plt
%matplotlib inline

from utils import config_gpu, img_to_array, load_img
_ = config_gpu()

/home/youjin2/venvs/py3/lib/python3.6/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## why LAB? 
- idea:
    - L represents luminance (gray-scale)
    - A, B represent color
- model structure
    - input: L
    - output: AB

## load and split data

In [2]:
data_path = '../data/chap9/img/colorize/'
data_lists = glob.glob(os.path.join(data_path, '*.jpg'))

In [4]:
val_n_sample = math.floor(len(data_lists)*0.1)
test_n_sample = math.floor(len(data_lists)*0.1)
train_n_sample = len(data_lists) - val_n_sample - test_n_sample

In [5]:
val_lists = data_lists[:val_n_sample]
test_lists = data_lists[val_n_sample:(val_n_sample+test_n_sample)]
train_lists = data_lists[(val_n_sample+test_n_sample):(train_n_sample+val_n_sample+test_n_sample)]

## preprocess RGB to LAB

In [6]:
img_size = 224

In [7]:
def rgb2lab(rgb):
    assert rgb.dtype == 'uint8'
    
    return cv.cvtColor(rgb, cv.COLOR_RGB2LAB)

def lab2rgb(lab):    
    assert lab.dtype == 'uint8'
    
    return cv.cvtColor(rgb, cv.COLOR_LAB2RGB)

def get_lab_from_data_list(data_list):
    
    x_lab = []
    for f in data_list:
        rgb = img_to_array(
            load_img(f, target_size=(img_size, img_size))
        ).astype(np.uint8)
        lab = rgb2lab(rgb)
        x_lab.append(lab)        
        
    return np.stack(x_lab)

def generator_with_preprocessing(data_list, batch_size, shuffle=False):
    
    while True:
        if shuffle:
            np.random.shuffle(data_list)
        for i in range(0, len(data_list), batch_size):
            batch_list = data_list[i:i+batch_size]
            batch_lab = get_lab_from_data_list(batch_list)
            batch_l = batch_lab[:, :, :, 0:1]
            batch_ab = batch_lab[:, :, :, 1:]
            yield (batch_l, batch_ab)

In [11]:
batch_size = 128
epochs = 100

In [12]:
train_gen = generator_with_preprocessing(train_lists, batch_size, shuffle=True)
val_gen = generator_with_preprocessing(val_lists, batch_size, shuffle=False)
test_gen = generator_with_preprocessing(test_lists, batch_size, shuffle=False)

In [13]:
train_steps = math.ceil(len(train_lists)/batch_size)
val_steps = math.ceil(len(val_lists)/batch_size)
test_steps = math.ceil(len(test_lists)/batch_size)

## build model

In [14]:
autoencoder = Sequential()

In [15]:
# encoder
# (224, 224, 1) -> (224, 224, 32)
autoencoder.add(
    Conv2D(
        32,
        (3, 3),
        (1, 1),
        activation='relu',
        padding='same',
        input_shape=(224, 224, 1)
    )
)
# (224, 224, 32) -> (112, 112, 64)
autoencoder.add(
    Conv2D(
        64,
        (3, 3),
        (2, 2),
        activation='relu',
        padding='same'
    )
)
# (112, 112, 64) -> (56, 56, 128)
autoencoder.add(
    Conv2D(
        128,
        (3, 3),
        (2, 2),
        activation='relu',
        padding='same'
    )
)
# (56, 56, 128) -> (28, 28, 256)
autoencoder.add(
    Conv2D(
        256,
        (3, 3),
        (2, 2),
        activation='relu',
        padding='same'
    )
)
# decoder
# (28, 28, 256) -> (56, 56, 128)
autoencoder.add(
    Conv2DTranspose(
        128,
        (3, 3),
        (2, 2),
        activation='relu',
        padding='same'
    )
)
# (56, 56, 128) -> (112, 112, 64)
autoencoder.add(
    Conv2DTranspose(
        64,
        (3, 3),
        (2, 2),
        activation='relu',
        padding='same'
    )
)
# (112, 112, 64) -> (224, 224, 32)
autoencoder.add(
    Conv2DTranspose(
        32,
        (3, 3),
        (2, 2),
        activation='relu',
        padding='same'
    )
)
# (224, 224, 32) -> (224, 224, 2)
autoencoder.add(
    Conv2D(
        2,
        (1, 1),
        (1, 1),
        activation='relu',
        padding='same'
    )
)

In [16]:
autoencoder.compile(optimizer='adam', loss='mse')

In [17]:
autoencoder.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 32)      320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 64)      18496     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 56, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 256)       295168    
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 56, 56, 128)       295040    
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 112, 112, 64)      73792     
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 224, 224, 32)      1

In [18]:
cp_path = '../model/autoencoder_colorize/'
cp_callback = ModelCheckpoint(cp_path, save_weights_only=True, verbose=0)

In [21]:
autoencoder.fit_generator(
    generator=train_gen,
    steps_per_epoch=train_steps,
    epochs=epochs,
    validation_data=val_gen,
    validation_steps=val_steps,
    callbacks=[cp_callback]
)

Epoch 1/100
184/184 [==============================] - 104s 564ms/step - loss: 1710.7228 - val_loss: 1892.7988
Epoch 2/100
 29/184 [===>..........................] - ETA: 1:24 - loss: 1502.1258

KeyboardInterrupt: 

In [ ]:
autoencoder.load_weights(cp_path)